# DAX Benchmarks

In [151]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sempy.fabric as fabric
import time

from plotly.subplots import make_subplots

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 46, Finished, Available, Finished)

## Parameters

In [152]:
WORKSPACE = None  # same workspace as notebook/lakehouse; set name/ID if needed

DATASETS = {
    "SS": "Star Schema",
    "USS": "Unified Star Schema",
}

WARMUP_RUNS = 2
MEASURED_RUNS = 10

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 47, Finished, Available, Finished)

## Helpers

In [153]:
# Global accumulator for all runs
GLOBAL_RUNS = []

# Fact table mapping
FACT_FOR_MEASURE_PREFIX = {
    "SS": {
        "fact": "fact__sales",
        "fact_ret": "fact__returns",
        "fact_inv": "fact__inventory",
    },
    "USS": {
        "fact": "_bridge",
        "fact_ret": "_bridge",
        "fact_inv": "_bridge",
    }
}

# Use Hex codes for Plotly compatibility (C0=Blue, C1=Orange)
DATASET_COLORS = {
    "Star Schema": "#1f77b4",
    "Unified Star Schema": "#ff7f0e"
}

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 48, Finished, Available, Finished)

In [154]:
def run_one_dax(dataset_name: str, dax: str, workspace=None) -> pd.DataFrame:
    return fabric.evaluate_dax(dataset=dataset_name, dax_string=dax, workspace=workspace)

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 49, Finished, Available, Finished)

In [155]:
def run_query(dataset_name: str, suite_key: str, group_name: str, qname: str, dax: str, workspace=None):
    """
    Runs warmups + measured repetitions.
    Stops immediately on error.
    """
    runs_rows_local = []
    seq = 0
    stop_reason = "completed"

    def record(run_phase: str, rep_n: int, duration_ms: float, df, err: str | None):
        nonlocal seq
        seq += 1
        row_count = None if df is None else int(len(df))
        col_count = None if df is None else int(df.shape[1])

        runs_rows_local.append({
            "dataset": dataset_name,
            "suite": suite_key,
            "query_group": group_name,
            "query_name": qname,
            "phase": run_phase,
            "repeat_n": rep_n,
            "seq_in_query": seq,
            "duration_ms": float(duration_ms),
            "row_count": row_count,
            "col_count": col_count,
            "error": err,
        })

    # ---- Warmups ----
    for w in range(1, WARMUP_RUNS + 1):
        t0 = time.perf_counter()
        df = None
        err = None
        try:
            df = run_one_dax(dataset_name, dax, workspace=workspace)
        except Exception as e:
            err = str(e)
        t1 = time.perf_counter()
        record("warmup", w, (t1 - t0) * 1000.0, df, err)
        
        if err:
            stop_reason = "error_in_warmup"
            runs_rows_local[-1]["stop_reason"] = stop_reason
            return runs_rows_local, stop_reason

    # ---- Measured ----
    for m in range(1, MEASURED_RUNS + 1):
        t0 = time.perf_counter()
        df = None
        err = None
        try:
            df = run_one_dax(dataset_name, dax, workspace=workspace)
        except Exception as e:
            err = str(e)
        t1 = time.perf_counter()
        record("measured", m, (t1 - t0) * 1000.0, df, err)

        if err:
            stop_reason = "error"
            runs_rows_local[-1]["stop_reason"] = stop_reason
            return runs_rows_local, stop_reason

    if runs_rows_local:
        runs_rows_local[-1]["stop_reason"] = stop_reason

    return runs_rows_local, stop_reason

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 50, Finished, Available, Finished)

In [ ]:
def run_and_plot(group, name, template):
    print(f"--- Running {name} ({group}) ---")
    local_runs = []
    
    for suite_key, dataset_name in DATASETS.items():
        mapping = FACT_FOR_MEASURE_PREFIX[suite_key]
        dax = template.format(**mapping)
        
        # Call simplified run_query
        rr, stop_reason = run_query(
            dataset_name=dataset_name,
            suite_key=suite_key,
            group_name=group,
            qname=name,
            dax=dax,
            workspace=WORKSPACE
        )
        local_runs.extend(rr)
        print(f"[{suite_key}] Stop reason: {stop_reason}")
        
        if stop_reason != "completed":
            if rr:
                last_error = rr[-1].get("error")
                if last_error:
                    print(f"[{suite_key}] ERROR: {last_error}")
            # Stop if the first dataset fails
            break
        
    # Append to global
    GLOBAL_RUNS.extend(local_runs)
    
    # Plot
    df = pd.DataFrame(local_runs)
    if not df.empty:
        measured = df.query("phase == 'measured' and error.isna()")
        if not measured.empty:
            stats = measured.groupby("dataset")["duration_ms"].median().reset_index()
            
            fig = px.bar(
                stats, 
                x="duration_ms", 
                y="dataset", 
                orientation='h', 
                title=name,
                color="dataset",
                color_discrete_map=DATASET_COLORS,
                text_auto='.0f'
            )
            
            fig.update_layout(
                height=250, 
                margin=dict(l=0, r=0, t=40, b=0),
                template="plotly_white",
                showlegend=False,
                xaxis_title="Median Duration (ms)",
                yaxis_title=None
            )
            fig.show()

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 51, Finished, Available, Finished)

## Queries

### Single Fact

#### SF01 - Sales By Year

In [157]:
run_and_plot("Single Fact", "SF01_sales_by_year", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
)
ORDER BY 'dim__date'[year]
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 52, Finished, Available, Finished)

--- Running SF01_sales_by_year (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF02 - Sales By Year Category

In [158]:
run_and_plot("Single Fact", "SF02_sales_by_year_category", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__item'[category],
    "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
)
ORDER BY 'dim__date'[year], 'dim__item'[category]
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 53, Finished, Available, Finished)

--- Running SF02_sales_by_year_category (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF03 - Sales Many Groups

In [159]:
run_and_plot("Single Fact", "SF03_sales_many_groups", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__item'[category],
    'dim__store'[_key__dim__store],
    "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 54, Finished, Available, Finished)

--- Running SF03_sales_many_groups (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF04 - Top 100 Items Sales 2002

In [160]:
run_and_plot("Single Fact", "SF04_top100_items_sales_2002", """
EVALUATE
TOPN(
    100,
    SUMMARIZECOLUMNS(
        'dim__item'[_key__dim__item],
        'dim__item'[category],
        TREATAS( {{ 2002 }}, 'dim__date'[year] ),
        "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
    ),
    [Sales], DESC,
    'dim__item'[_key__dim__item], ASC
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 55, Finished, Available, Finished)

--- Running SF04_top100_items_sales_2002 (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF05 - Inventory WH Category Year 2002

In [161]:
run_and_plot("Single Fact", "SF05_inventory_wh_category_year_2002", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__warehouse'[_key__dim__warehouse],
    'dim__item'[category],
    TREATAS( {{ 2002 }}, 'dim__date'[year] ),
    "Qty On Hand", SUM('{fact_inv}'[_measure__inventory__qty_on_hand])
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 56, Finished, Available, Finished)

--- Running SF05_inventory_wh_category_year_2002 (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF06 - Sales YoY By Year

In [162]:
run_and_plot("Single Fact", "SF06_sales_yoy_by_year", """
EVALUATE
ADDCOLUMNS(
    SUMMARIZECOLUMNS(
        'dim__date'[year],
        "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
    ),
    "Sales PY",
        CALCULATE(
            SUM('{fact}'[_measure__sales__ext_sales_price]),
            SAMEPERIODLASTYEAR('dim__date'[date])
        ),
    "YoY Δ",
        SUM('{fact}'[_measure__sales__ext_sales_price])
        - CALCULATE(
            SUM('{fact}'[_measure__sales__ext_sales_price]),
            SAMEPERIODLASTYEAR('dim__date'[date])
        ),
    "YoY %",
        DIVIDE(
            SUM('{fact}'[_measure__sales__ext_sales_price])
            - CALCULATE(
                SUM('{fact}'[_measure__sales__ext_sales_price]),
                SAMEPERIODLASTYEAR('dim__date'[date])
            ),
            CALCULATE(
                SUM('{fact}'[_measure__sales__ext_sales_price]),
                SAMEPERIODLASTYEAR('dim__date'[date])
            )
        )
)
ORDER BY 'dim__date'[year]
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 57, Finished, Available, Finished)

--- Running SF06_sales_yoy_by_year (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF07 - Distinct Customers With Orders

In [163]:
run_and_plot("Single Fact", "SF07_distinct_customers_with_orders", """
EVALUATE
ROW(
    "Distinct Customers With Orders",
    CALCULATE(DISTINCTCOUNT('{fact}'[_key__dim__customer]), NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price]))
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 58, Finished, Available, Finished)

--- Running SF07_distinct_customers_with_orders (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF08 - Sales Hi Card Group

In [164]:
run_and_plot("Single Fact", "SF08_sales_hi_card_group", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__date'[month_of_year],
  'dim__item'[category],
  'dim__store'[_key__dim__store],
  "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 59, Finished, Available, Finished)

--- Running SF08_sales_hi_card_group (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF09 - Top 500 Items By Year

In [165]:
run_and_plot("Single Fact", "SF09_top500_items_by_year", """
EVALUATE
TOPN(
  500,
  SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__item'[_key__dim__item],
    'dim__item'[category],
    "Sales", SUM('{fact}'[_measure__sales__ext_sales_price])
  ),
  [Sales], DESC,
  'dim__date'[year], ASC,
  'dim__item'[_key__dim__item], ASC
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 60, Finished, Available, Finished)

--- Running SF09_top500_items_by_year (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF10 - Sales Rolling 90d By Category

In [166]:
run_and_plot("Single Fact", "SF10_sales_rolling_90d_by_category", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__date'[month_of_year],
  'dim__item'[category],
  "Sales 90D",
    CALCULATE(
      SUM('{fact}'[_measure__sales__ext_sales_price]),
      DATESINPERIOD('dim__date'[date], MAX('dim__date'[date]), -90, DAY)
    )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 61, Finished, Available, Finished)

--- Running SF10_sales_rolling_90d_by_category (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF11 - New Customers YoY

In [167]:
run_and_plot("Single Fact", "SF11_new_customers_yoy", """
EVALUATE
VAR CustThisYear =
  CALCULATETABLE(
    VALUES('{fact}'[_key__dim__customer]),
    NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price])
  )
VAR CustPriorYear =
  CALCULATETABLE(
    VALUES('{fact}'[_key__dim__customer]),
    SAMEPERIODLASTYEAR('dim__date'[date]),
    NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price])
  )
RETURN
ROW(
  "New Customers (YoY)",
  COUNTROWS( EXCEPT(CustThisYear, CustPriorYear) )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 62, Finished, Available, Finished)

--- Running SF11_new_customers_yoy (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF12 - Distinct Customers Hi Card

In [168]:
run_and_plot("Single Fact", "SF12_distinct_customers_hi_card", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__item'[category],
  'dim__store'[_key__dim__store],
  "Distinct Customers",
    CALCULATE(
      DISTINCTCOUNT('{fact}'[_key__dim__customer]),
      NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price])
    )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 63, Finished, Available, Finished)

--- Running SF12_distinct_customers_hi_card (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### SF13 - Hi Card 30d Distinct Customers

In [169]:
run_and_plot("Single Fact", "SF13_hi_card_30d_distinct_customers", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__item'[_key__dim__item],
  "Distinct Customers 30D",
    CALCULATE(
      DISTINCTCOUNT('{fact}'[_key__dim__customer]),
      DATESINPERIOD('dim__date'[date], MAX('dim__date'[date]), -30, DAY)
    )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 64, Finished, Available, Finished)

--- Running SF13_hi_card_30d_distinct_customers (Single Fact) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


### Multi Fact Aggregations

#### MF01 - Net Sales By Month

In [170]:
run_and_plot("Multi Fact Aggregations", "MF01_net_sales_by_month", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__date'[month_of_year],
    "Sales",   SUM('{fact}'[_measure__sales__ext_sales_price]),
    "Returns", SUM('{fact_ret}'[_measure__returns__return_amt]),
    "Net",
        SUM('{fact}'[_measure__sales__ext_sales_price])
        - SUM('{fact_ret}'[_measure__returns__return_amt])
)
ORDER BY 'dim__date'[year], 'dim__date'[month_of_year]
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 65, Finished, Available, Finished)

--- Running MF01_net_sales_by_month (Multi Fact Aggregations) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### MF02 - Return Rate By Year Category

In [171]:
run_and_plot("Multi Fact Aggregations", "MF02_return_rate_by_year_category", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__item'[category],
  "Sales",   SUM('{fact}'[_measure__sales__ext_sales_price]),
  "Returns", SUM('{fact_ret}'[_measure__returns__return_amt]),
  "Return Rate",
    DIVIDE(
      SUM('{fact_ret}'[_measure__returns__return_amt]),
      SUM('{fact}'[_measure__sales__ext_sales_price])
    )
)
ORDER BY 'dim__date'[year], 'dim__item'[category]
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 66, Finished, Available, Finished)

--- Running MF02_return_rate_by_year_category (Multi Fact Aggregations) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### MF03 - Two Fact Separate Aggregates

In [172]:
run_and_plot("Multi Fact Aggregations", "MF03_two_fact_separate_aggregates", """
EVALUATE
SUMMARIZECOLUMNS(
  'dim__date'[year],
  'dim__store'[_key__dim__store],
  "Sales (all items)",
    CALCULATE(
      SUM('{fact}'[_measure__sales__ext_sales_price]),
      REMOVEFILTERS('dim__item')
    ),
  "Returns (all reasons)",
    CALCULATE(
      SUM('{fact_ret}'[_measure__returns__return_amt]),
      REMOVEFILTERS('dim__reason')
    ),
  "Return Rate",
    DIVIDE(
      CALCULATE(
        SUM('{fact_ret}'[_measure__returns__return_amt]),
        REMOVEFILTERS('dim__reason')
      ),
      CALCULATE(
        SUM('{fact}'[_measure__sales__ext_sales_price]),
        REMOVEFILTERS('dim__item')
      )
    )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 67, Finished, Available, Finished)

--- Running MF03_two_fact_separate_aggregates (Multi Fact Aggregations) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


### Cross Fact Set Operations

#### CF01 - Distinct Customers Sales Or Returns

In [173]:
run_and_plot("Cross Fact Set Ops", "CF01_distinct_customers_sales_or_returns", """
EVALUATE
VAR CustSales =
  CALCULATETABLE(
    VALUES('{fact}'[_key__dim__customer]),
    NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price])
  )
VAR CustReturns =
  CALCULATETABLE(
    VALUES('{fact_ret}'[_key__dim__customer]),
    NOT ISBLANK('{fact_ret}'[_measure__returns__return_amt])
  )
RETURN
ROW(
  "Distinct Customers (Sales OR Returns)",
  COUNTROWS( DISTINCT( UNION(CustSales, CustReturns) ) )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 68, Finished, Available, Finished)

--- Running CF01_distinct_customers_sales_or_returns (Cross Fact Set Ops) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


#### CF02 - Customers Sales And Returns

In [174]:
run_and_plot("Cross Fact Set Ops", "CF02_customers_sales_and_returns", """
EVALUATE
VAR CustSales =
  CALCULATETABLE(
    VALUES('{fact}'[_key__dim__customer]),
    NOT ISBLANK('{fact}'[_measure__sales__ext_sales_price])
  )
VAR CustReturns =
  CALCULATETABLE(
    VALUES('{fact_ret}'[_key__dim__customer]),
    NOT ISBLANK('{fact_ret}'[_measure__returns__return_amt])
  )
RETURN
ROW(
  "Customers (Sales AND Returns)",
  COUNTROWS( INTERSECT(CustSales, CustReturns) )
)
""")

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 69, Finished, Available, Finished)

--- Running CF02_customers_sales_and_returns (Cross Fact Set Ops) ---
[SS] Stop reason: completed
[USS] Stop reason: completed


## Summarize

In [175]:
# Prepare data
filtered_df = pd.DataFrame(GLOBAL_RUNS).query("phase == 'measured' and error.isna()")

# 1. Median Duration by Dataset
dataset_stats = filtered_df.groupby("dataset")["duration_ms"].median().reset_index()
fig1 = px.bar(
    dataset_stats,
    x="duration_ms",
    y="dataset",
    orientation='h',
    title="Median Duration By Dataset",
    color="dataset",
    color_discrete_map=DATASET_COLORS,
    text_auto='.0f',
    labels={"duration_ms": "Median Duration (ms)", "dataset": "Dataset"}
)
fig1.update_layout(template="plotly_white", showlegend=False, height=300)
fig1.show()

# 2. Median Duration by Query Group
group_stats = filtered_df.groupby(["query_group", "dataset"])["duration_ms"].median().reset_index()
fig2 = px.bar(
    group_stats,
    x="duration_ms",
    y="query_group",
    color="dataset",
    barmode="group",
    orientation='h',
    title="Median Duration By Query Group",
    color_discrete_map=DATASET_COLORS,
    text_auto='.0f',
    labels={"duration_ms": "Median Duration (ms)", "query_group": "Query Group"}
)
fig2.update_layout(
    template="plotly_white", 
    height=400,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        title=""
    )
)
fig2.show()

# 3. Detailed Charts per Group
groups = ["Single Fact", "Multi Fact Aggregations", "Cross Fact Set Ops"]

for group_name in groups:
    group_df = filtered_df.query(f"query_group == '{group_name}'").groupby(["query_name", "dataset"])["duration_ms"].median().reset_index()
    
    if not group_df.empty:
        fig = px.bar(
            group_df,
            x="query_name",
            y="duration_ms",
            color="dataset",
            barmode="group",
            title=f"Median Duration: {group_name}",
            color_discrete_map=DATASET_COLORS,
            labels={"duration_ms": "Median Duration (ms)", "query_name": "Query Name"}
        )
        fig.update_layout(
            template="plotly_white", 
            xaxis_tickangle=-45,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                title=""
            )
        )
        fig.show()

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 70, Finished, Available, Finished)

## Environment

In [176]:
# Install watermark if not already installed
try:
    import watermark
except ImportError:
    %pip -q install watermark

%load_ext watermark
%watermark -n -u -v -iv -w

StatementMeta(, 9b8bda24-9a8c-4910-a4ea-a4ab43d88216, 71, Finished, Available, Finished)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Mon, 29 Dec 2025

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.20.0

notebookutils: 1.1.12.35
pandas       : 2.1.4
plotly       : 5.22.0
pyspark      : 3.5.1.5.4.20240407
sempy        : 0.11.0
watermark    : 2.6.0

Watermark: 2.6.0

